In [4]:
from IslandTime import retrieve_island_info, Segmentation, update_results_map, plot_shoreline_transects, Workflow
import pandas as pd
import os
from tqdm import tqdm
import shapely
import geopandas as gpd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import skimage.exposure as exposure
import pyproj

def _image_stretch(im):
    # Rescale image for visualisation
    if np.ndim(im) > 2:
        for i in range(np.shape(im)[2]):
            im_no_nan = im[:, :, i][~np.isnan(im[:, :, i])]
            im[:, :, i] = exposure.rescale_intensity(im[:, :, i], in_range=(im_no_nan.min(), im_no_nan.max()))
    
    else:
        im_no_nan = im[~np.isnan(im)]
        im = exposure.rescale_intensity(im, in_range=(im_no_nan.min(), im_no_nan.max()))
    
    return im

rgb = all_dict[1]['26-08-2015_S2']
georef = all_dict[2]['26-08-2015_S2']

fig, ax = plt.subplots()

ax.imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])
plot_shoreline_transects(island_info, transect_plot=10, ax=ax)
ax.axis('off')

(280860.0, 281410.0, 583140.0, 583680.0)

In [136]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from matplotlib.collections import PolyCollection, LineCollection

t = 10
ts = island_info['timeseries_preprocessing']['optimal time period']['dict_timeseries']['coastline_position_transect_{}_waterline'.format(t)]['monthly']['coastline_position_transect_{}_waterline'.format(t)]

def plot_acf_colors(ax, markercolor="firebrick", linecolor="black", facecolor="silver", barcolor="darkcyan", linewidth=1):
    """utility function to get some control over colors with  plot_acf()"""
    
    for item in ax.collections:
        # change the color of the confidence interval 
        if type(item) == PolyCollection:
            item.set_facecolor(facecolor)
        # change the color of the vertical lines
        if type(item) == LineCollection:
            item.set_color(barcolor)
    # change the color of the markers
    [line.get_label() for line in ax.lines]
    for item in ax.lines:
        item.set_color(markercolor)
    # change the color of the horizontal lines
    ax.lines[0].set_color(linecolor)
    ax.lines[0].set_linewidth(linewidth)
    #ax.lines.remove(ax.lines[0])
    return ax

fig, axs = plt.subplots(2,2, figsize=(20,20))
axs = axs.ravel()
plot_acf(ts, lags=40, alpha=0.05, ax=axs[2], color='k', use_vlines=True, fft=True, vlines_kwargs={'colors': 'k', 'linestyles': 'dashed'}, title='', gapcolor='k')
ax = plot_acf_colors(axs[2])
axs[2].axvline(x=12, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
axs[2].axvline(x=6, color='k', linestyle='-', linewidth=0.5, zorder=0, alpha=0.5)
# ax.axvline(x=24, color='red', linestyle='--')
# ax.axvline(x=36, color='red', linestyle='--')
# ax.axvline(x=6, color='g', linestyle='--', label='6 months')
# ax.axvline(x=18, color='g', linestyle='--')
# ax.axvline(x=30, color='g', linestyle='--')
axs[2].set_xlabel('Lag (months)', fontsize=15)
axs[2].set_ylabel('Autocorrelation', fontsize=15)
axs[2].text(12.5, 0.8, '12 months', fontsize=10, fontweight='bold')
axs[2].text(6.5, 0.8, '6 months', fontsize=10, fontweight='bold')
axs[2].set_xlim(-1, 40.5)
axs[2].set_ylim(-1.05, 1.05)

axs[1].plot(ts, color='k', linewidth=1.5, label='Preprocessed time series')
axs[1].set_xlabel('Time', fontsize=15)
axs[1].set_ylabel('Coastline position (m)', fontsize=15)
axs[1].legend()
# plot_shoreline_transects(island_info, ax=axs[0], transect_plot=10)

axs[3].plot(ts, color='k', linewidth=1.5, alpha=0.3)
axs[3].set_xlabel('Time', fontsize=15)
axs[3].set_ylabel('Coastline position (m)', fontsize=15)
# ax.set_title('')
# ax.legend(fontsize=15)

import numpy as np
import skimage.exposure as exposure
import pyproj

def _image_stretch(im):
    # Rescale image for visualisation
    if np.ndim(im) > 2:
        for i in range(np.shape(im)[2]):
            im_no_nan = im[:, :, i][~np.isnan(im[:, :, i])]
            im[:, :, i] = exposure.rescale_intensity(im[:, :, i], in_range=(im_no_nan.min(), im_no_nan.max()))
    
    else:
        im_no_nan = im[~np.isnan(im)]
        im = exposure.rescale_intensity(im, in_range=(im_no_nan.min(), im_no_nan.max()))
    
    return im

kk = 3
georef = all_dict[2][list(all_dict[2].keys())[kk]]
rgb = all_dict[1][list(all_dict[1].keys())[kk]]

axs[0].imshow(_image_stretch(rgb), extent=[georef[0], georef[0] + georef[1] * rgb.shape[1], georef[3] + georef[5] * rgb.shape[0], georef[3]])
plot_shoreline_transects(island_info, transect_plot=t, ax=axs[0])
axs[0].axis('off')

from scipy.fft import fft
from scipy.signal import find_peaks, argrelextrema
from scipy import stats


time_series = ts
# Fourier transform of time series
fft_result = fft(time_series.values)

# Get frequencies
frequencies = np.fft.fftfreq(len(time_series.values), 1/12)

# fft_result without frequency 0
fft_result_n = fft_result[1:]

# Remove frequencies with amplitude below a threshold
#print(all(element is False for element in (stats.zscore(np.abs(fft_result_n)) > self.z_score_seasonality)))
score_list = stats.zscore(np.abs(fft_result_n)) > 2
if all(not element for element in score_list):
    fft_result = fft_result
else:
    threshold = np.min(np.abs(fft_result_n[stats.zscore(np.abs(fft_result_n)) > 2]))
    fft_result[np.abs(fft_result) < threshold] = 0

# Inverse Fourier transform (filetered data)
filtered_data_fourier = np.fft.ifft(fft_result).real

# Find peaks
peaks_fourier, _ = find_peaks(filtered_data_fourier, height=0)

# Find period
period_fourier = stats.mode(np.diff(peaks_fourier)).mode

axs[3].plot(time_series.index, filtered_data_fourier, color='seagreen', linewidth=1.5, label='Fourier-filtered time series')
ii=0
for peak in peaks_fourier:
        if ii==0:
            axs[3].axvline(time_series.index[peak], color='firebrick', linestyle='--', linewidth=0.8, label='Signal peaks')
        else:
            axs[3].axvline(time_series.index[peak], color='firebrick', linestyle='--', linewidth=0.8)
        ii+=1

import datetime
# axs[3].annotate('', xy=(datetime.datetime(2017, 5, 1), 136), xytext=(datetime.datetime(2018, 4, 1), 136), arrowprops=dict(arrowstyle='<->', color='black'))
# plt.text(datetime.datetime(2017, 10, 15), 138, '12 months', horizontalalignment='center', verticalalignment='center', fontsize=10, fontweight='bold')
axs[3].legend(fontsize=10)
# ax[1, 1].set_title('Fourier reconstruction'

fig.savefig('figures//seasonal_signal.png', dpi=300)

# axs[0].text()

In [163]:
from statsmodels.tsa.seasonal import STL
import Rbeast as rb

res = STL(time_series, period=12, seasonal=13).fit()
res_B = rb.beast(ts.values, start=[ts.index[0].year, ts.index[0].month, ts.index[0].day], season='harmonic', deltat='1/12 year', period='1 year', quiet=True, print_progress=False)
# rb.plot(res_B)
# res.plot()

fig, ax = plt.subplots(figsize=(20,20))

ax.plot(ts.index, res.seasonal, color='seagreen', label='STL', linewidth=2)
ax.plot(ts.index, res_B.season.Y, color='firebrick', label='BEAST', linewidth=2)
ax.fill_between(ts.index, res_B.season.CI[:, 0], res_B.season.CI[:, 1], alpha=0.3, color='firebrick')
ax.fill_between(ts.index, res.seasonal - res.seasonal, res.seasonal + res.seasonal, color='seagreen', alpha=0.3)
ax.set_xlim(ts.index[0], ts.index[-1])
ax.legend(fontsize=20)
ax.set_xlabel('Time', fontsize=15)
ax.set_ylabel('Coastline time series seasonal component amplitude (m)', fontsize=15)

Text(0, 0.5, 'Coastline time series seasonal component amplitude (m)')

In [142]:
falig, ax = plt.subplots()
# plt.imshow(im)
ax.plot(np.arange(10), np.sin(np.arange(10)))
ax.annotate('', xy=(0, 0), xytext=(5, 0.2),
             arrowprops=dict(arrowstyle='<->', color='black'))

Text(5, 0.2, '')

In [16]:
# Read file for islands to ignore
import shapely.geometry


df_islands_ignore = pd.read_excel('islands_to_ignore.xlsx')
list_islands_ignore = df_islands_ignore['Island'].values

path_to_data = os.path.join(os.getcwd(), 'data', 'info_islands')

print('Updating results maps...')

multi_point = []
multi_point_excluded = []

for file in tqdm(os.listdir(path_to_data)):
    island = file.split('_')[1] 
    country = file.split('_')[2].split('.')[0]

    try:
        island_info = retrieve_island_info(island, country, verbose=False)
        lat, lon = island_info['spatial_reference']['latitude'], island_info['spatial_reference']['longitude']
        point = shapely.geometry.Point(lon, lat)

        if island in list_islands_ignore:
            label = 'excluded'
            multi_point_excluded.append(point)

        else:
            label = 'included'
            multi_point.append(point)
    
    except:
        print('Error with island:', island)
        continue

mp = shapely.geometry.MultiPoint(multi_point)
mp_excluded = shapely.geometry.MultiPoint(multi_point_excluded)
gdf = gpd.GeoDataFrame(geometry=[mp], crs='EPSG:4326')
gdf_excluded = gpd.GeoDataFrame(geometry=[mp_excluded], crs='EPSG:4326')
gdf.to_file('shp//Huvadhoo_included.shp')
gdf_excluded.to_file('shp//Huvadhoo_excluded.shp')

Updating results maps...


  0%|          | 0/401 [00:00<?, ?it/s]

 86%|████████▋ | 346/401 [01:19<00:10,  5.46it/s]

Error with island: Qaruh Island


100%|██████████| 401/401 [01:29<00:00,  4.49it/s]
